---
## <span style="color:orange"> Host Multiple TensorFlow Computer Vision Models using SageMaker Multi-Model Endpoint </span>
---
## <span style="color:black">Contents</span>
1. [Background](#Background)
1. [Setup](#Setup)
1. [Train Model 2 - Sign Language Image Classification](#Train-Model-2---Sign-Language-Image-Classification)
1. [Create a Multi-Model Endpoint](#Create-a-Multi-Model-Endpoint)
1. [Test Multi-Model Endpoint for Real Time Inference](#Test-Multi-Model-Endpoint-for-Real-Time-Inference)

### Background
In this notebook, we show how to host multiple computer vision models trained using the TensorFlow framework under one SageMaker multi-model endpoint.  For the model we use a pretrained VGG16 CNN model pretrained on the ImageNet dataset and fine-tune on Sign Language Digits Dataset.

in this notebook we are using the final version as training is not the focus. To simulate multiple models we will create copies of the same model and load them in S3

### Setup

 #### Prerequisites 
 Choose Kernel for this notebook.<br>
 Under `Kernel` tab at the top of this notebook &#8594; `Choose kernel`, select `conda_python3` 

In [ ]:
%%capture

!pip install tensorflow==2.3.0

In [ ]:
!pip uninstall redis -y
!pip uninstall redis-py-cluster -y


In [ ]:
!pip install redis-py-cluster

In [9]:
!pip install boto3

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/mxnet_p37/bin/python -m pip install --upgrade pip' command.


#### Imports 

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sagemaker.tensorflow.serving import TensorFlowModel
from sagemaker.multidatamodel import MultiDataModel
from tensorflow.keras.datasets import cifar10
from sagemaker.tensorflow import TensorFlow
from sagemaker.inputs import TrainingInput
from sagemaker import get_execution_role
from tensorflow.keras import utils
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from datetime import datetime
import tensorflow as tf
import numpy as np
import sagemaker
import logging
import boto3
import time
import os

2022-11-10 22:11:33.551547: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.0/lib64:/usr/local/cuda-11.0/extras/CUPTI/lib64:/usr/local/cuda-11.0/lib:/usr/local/cuda-11.0/efa/lib:/opt/amazon/efa/lib:/opt/amazon/efa/lib64:/opt/amazon/efa/lib64:/opt/amazon/openmpi/lib64:/usr/local/lib:/usr/lib:/lib:
2022-11-10 22:11:33.551577: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import redis
from rediscluster import RedisCluster

### Using sageMaker VPC

In [4]:
from redis import Redis
import logging
redis_client = Redis(
    host='testredis.m7ovi1.ng.0001.use1.cache.amazonaws.com', 
    port=6379, #decode_responses=True, ssl=True, 
    username='default', #'testredisuser', 'default'
    #password='testRedisUserPassword'
)
# redis_client = Redis(
#     host='testredis.m7ovi1.ng.0001.use1.cache.amazonaws.com',
#     username='testredisuser',
#     password='testRedisUserPassword', ssl_cert_reqs=None,  decode_responses=True)

if redis_client.ping():
    logging.info("Connected to Redis")


INFO:root:Connected to Redis


#### Setup Logger

In [5]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())

In [6]:
logger.info(f'[Using TensorFlow version: {tf.__version__}]')
logger.info(f'[Using SageMaker version: {sagemaker.__version__}]')

[Using TensorFlow version: 2.3.0]
INFO:__main__:[Using TensorFlow version: 2.3.0]
[Using SageMaker version: 2.116.0]
INFO:__main__:[Using SageMaker version: 2.116.0]


#### Seed for Reproducability

In [7]:
SEED = 123
np.random.seed(SEED)
tf.random.set_seed(SEED)

#### Create Roles, Sessions and Data Locations

In [8]:
import sagemaker

role = sagemaker.get_execution_role()
session = boto3.Session()
sagemaker_session = sagemaker.Session()
region=sagemaker_session.boto_region_name
print(region)
account_id = sagemaker_session.account_id()
print(account_id)

s3 = session.resource('s3')
TF_FRAMEWORK_VERSION = '2.3.0'
BUCKET = sagemaker.Session().default_bucket()
PREFIX = 'cv-models'

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


us-east-1
622343165275


#### b) Data Exploration

In [ ]:
# - ./data/sign_language/train/0/IMG_1118.JPG
train_path  = './data/sign_language/train'
img = mpimg.imread(f'{train_path}/0/IMG_1118.JPG')
plt.imshow(img)

In [ ]:
img.shape

In [ ]:
!pip install -U awscli --quiet
!pip install torch==1.8.0 --quiet 
!pip install torchvision==0.9.0 --quiet

### Create custom container

In [68]:
%%sh

# The name of our algorithm
algorithm_name=cpu-cv-redis

cd container

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -q -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
sha256:abb97409da1b0592ad722e43c12753d3e8dd4f2dbf7b63d5a670a63135082e92
The push refers to repository [622343165275.dkr.ecr.us-east-1.amazonaws.com/cpu-cv-redis]
6531ae424295: Preparing
148c995515a0: Preparing
dda72c5650f0: Preparing
87dae351871c: Preparing
f1a39e103d35: Preparing
0f13e18a9cc4: Preparing
b7abf7d0bb2c: Preparing
eba15308a6ba: Preparing
a92e3c51a0b6: Preparing
419e1b612781: Preparing
599518d61809: Preparing
dda6665a48d3: Preparing
69f57fbceb1b: Preparing
0f13e18a9cc4: Waiting
b7abf7d0bb2c: Waiting
eba15308a6ba: Waiting
a92e3c51a0b6: Waiting
419e1b612781: Waiting
599518d61809: Waiting
dda6665a48d3: Waiting
69f57fbceb1b: Waiting
6531ae424295: Layer already exists
dda72c5650f0: Layer already exists
148c995515a0: Layer already exists
87dae351871c: Layer already exists
f1a39e103d35: Layer already exists
b7abf7d0bb2c: Layer already exists
a92e3c51a0b6: Layer already exists
0f13e18a9cc4: Layer already exists
419e1b612781: Layer already exists
eba15308a6ba: Layer

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



## Download resnet model

In [10]:
import mxnet as mx
import os
import tarfile


In [48]:
import mxnet as mx
import os
import tarfile

model_path = "http://data.mxnet.io/models/imagenet/"
!mkdir -p models/resnet_18

mx.test_utils.download(
    model_path + "resnet/18-layers/resnet-18-0000.params", None, "models/resnet_18"
)
mx.test_utils.download(
    model_path + "resnet/18-layers/resnet-18-symbol.json", None, "models/resnet_18"
)
mx.test_utils.download(model_path + "synset.txt", None, "data/resnet_18")

with open("models/resnet_18/resnet-18-shapes.json", "w") as file:
    file.write('[{"shape": [1, 3, 224, 224], "name": "data"}]')

with tarfile.open("models/resnet_18.tar.gz", "w:gz") as tar:
    tar.add("models/resnet_18", arcname=".")
    
 
!mkdir -p models/resnet_152
mx.test_utils.download(
    model_path + "resnet/152-layers/resnet-152-0000.params", None, "models/resnet_152"
)
mx.test_utils.download(
    model_path + "resnet/152-layers/resnet-152-symbol.json", None, "models/resnet_152"
)
mx.test_utils.download(model_path + "synset.txt", None, "data/resnet_152")

with open("models/resnet_152/resnet-152-shapes.json", "w") as file:
    file.write('[{"shape": [1, 3, 224, 224], "name": "data"}]')

with tarfile.open("models/resnet_152.tar.gz", "w:gz") as tar:
    tar.add("data/resnet_152", arcname=".") 
    


INFO:root:models/resnet_18/resnet-18-0000.params exists, skipping download
INFO:root:models/resnet_18/resnet-18-symbol.json exists, skipping download
INFO:root:data/resnet_18/synset.txt exists, skipping download
INFO:root:models/resnet_152/resnet-152-0000.params exists, skipping download
INFO:root:models/resnet_152/resnet-152-symbol.json exists, skipping download
INFO:root:data/resnet_152/synset.txt exists, skipping download


In [11]:
import sagemaker
import boto3
sess = sagemaker.Session()
s3_bucket = sess.default_bucket()  # Replace with your own bucket name if needed
prefix = "cv-models"
print(s3_bucket)
print(prefix)

sm_client = boto3.client(service_name="sagemaker")
runtime_sm_client = boto3.client(service_name="sagemaker-runtime")

account_id = boto3.client("sts").get_caller_identity()["Account"]
region = boto3.Session().region_name

role = sagemaker.get_execution_role()
print(role)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


sagemaker-us-east-1-622343165275
cv-models
arn:aws:iam::622343165275:role/service-role/AmazonSageMaker-ExecutionRole-20220208T115633


In [50]:
import boto3
from botocore.client import ClientError
import os

s3 = boto3.resource("s3")
s3.meta.client.head_bucket(Bucket=s3_bucket)

models = {"resnet_18.tar.gz", "resnet_152.tar.gz"}

for model in models:
    key = os.path.join(prefix, model)
    with open("models/" + model, "rb") as file_obj:
        s3.Bucket(s3_bucket).Object(key).upload_fileobj(file_obj)    

### Create end point

In [69]:
from time import gmtime, strftime

model_name = "cpu-cv-redis" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
#model_url = "https://s3-{}.amazonaws.com/{}/{}/".format(region, s3_bucket, prefix)

    
container = "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(
    account_id, region, "cpu-cv-redis"
)

# print("Model name: " + model_name)
# print("Model data Url: " + model_url)
print("Container image: " + container)

#primary_container = {"Image": container, "ModelDataUrl": model_url, "Mode": "MultiModel"}

# create_model_response = sm_client.create_model(
#     ModelName=model_name, ExecutionRoleArn=role, Containers=[primary_container]
# )


vpc = 'vpc-05edeb4f9b293161c'
subnet_a = 'subnet-0508539ff391bc62a'
subnet_b = 'subnet-0f88fe2e674a870c4'

subnet_c = 'subnet-02e4d3f4bd7ac9e66'
subnet_d = 'subnet-0814f48bf38ffc0ae'
subnet_e = 'subnet-076597677e5d1293b'
subnet_f = 'subnet-09e3b111fe0bc7fa7'


security_group = 'sg-0394e815564b9f05a'

model_url = 's3://sagemaker-us-east-1-622343165275/cv-models/resnet_18.tar.gz'
print("Model name: " + model_name)
print("Model data Url: " + model_url)
print("Container image: " + container)

create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = {
        'Image': container,
        'ModelDataUrl': model_url
    },
    VpcConfig = {
        'SecurityGroupIds': [security_group],
        'Subnets': [subnet_a, subnet_b,subnet_c,subnet_d,subnet_e,subnet_f,],
    },
    EnableNetworkIsolation=False
)



print("Model Arn: " + create_model_response["ModelArn"])

Container image: 622343165275.dkr.ecr.us-east-1.amazonaws.com/cpu-cv-redis:latest
Model name: cpu-cv-redis2022-11-10-22-57-18
Model data Url: s3://sagemaker-us-east-1-622343165275/cv-models/resnet_18.tar.gz
Container image: 622343165275.dkr.ecr.us-east-1.amazonaws.com/cpu-cv-redis:latest
Model Arn: arn:aws:sagemaker:us-east-1:622343165275:model/cpu-cv-redis2022-11-10-22-57-18


#### End point config

In [70]:
endpoint_config_name = model_name
print("Endpoint config name: " + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.m5.xlarge",
            "InitialInstanceCount": 1,
            "InitialVariantWeight": 1,
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

Endpoint config name: cpu-cv-redis2022-11-10-22-57-18
Endpoint config Arn: arn:aws:sagemaker:us-east-1:622343165275:endpoint-config/cpu-cv-redis2022-11-10-22-57-18


### Create End point 

In [71]:
import time

endpoint_name = model_name
print("Endpoint name: " + endpoint_name)

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])



Endpoint name: cpu-cv-redis2022-11-10-22-57-18
Endpoint Arn: arn:aws:sagemaker:us-east-1:622343165275:endpoint/cpu-cv-redis2022-11-10-22-57-18


In [72]:
import time
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Endpoint Status: " + status)

print("Waiting for {} endpoint to be in service...".format(endpoint_name))
while 'Creating' == status:
    print(f"endpoint{endpoint_name}:: still creating...")
    time.sleep(30) # 30 sec
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    
#waiter = sm_client.get_waiter("endpoint_in_service")
#waiter.wait(EndpointName=endpoint_name)

print("Endpoint Created:::")

Endpoint Status: Creating
Waiting for cpu-cv-redis2022-11-10-22-57-18 endpoint to be in service...
endpointcpu-cv-redis2022-11-10-22-57-18:: still creating...
endpointcpu-cv-redis2022-11-10-22-57-18:: still creating...
endpointcpu-cv-redis2022-11-10-22-57-18:: still creating...
endpointcpu-cv-redis2022-11-10-22-57-18:: still creating...
endpointcpu-cv-redis2022-11-10-22-57-18:: still creating...
Endpoint Created:::


#### Invoke end point

In [73]:
fname = mx.test_utils.download(
    "https://github.com/dmlc/web-data/blob/master/mxnet/doc/tutorials/python/predict_image/cat.jpg?raw=true",
    "cat.jpg",
)

with open(fname, "rb") as f:
    payload = f.read()

INFO:root:cat.jpg exists, skipping download


In [74]:
print(type(payload))
print(len(payload))

<class 'bytes'>
227791


### Single model 

In [77]:
%%time

import json

response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/x-image",
    Body=payload,
)

print(*json.loads(response["Body"].read()), sep="\n")

probability=0.244390, class=label::277
probability=0.170342, class=label::278
probability=0.145019, class=label::263
probability=0.059833, class=label::335
probability=0.051555, class=label::282
CPU times: user 11.6 ms, sys: 3.53 ms, total: 15.1 ms
Wall time: 281 ms


In [79]:
import numpy as np

print("Starting invocation for CPU end point calling the GPU end point please wait ...")
results = []
for i in range(0, 1000):
    start = time.time()
    response = runtime_sm_client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType="application/x-image",
        Body=payload,
    )
    results.append((time.time() - start) * 1000)
print("\nPredictions for model latency: \n")
print("\nP95: " + str(np.percentile(results, 95)) + " ms\n")
print("P90: " + str(np.percentile(results, 90)) + " ms\n")
print("Average: " + str(np.average(results)) + " ms\n")

Starting invocation for CPU end point calling the GPU end point please wait ...

Predictions for model latency: 


P95: 108.73383283615112 ms

P90: 103.47027778625488 ms

Average: 95.71578693389893 ms



### Multi Model invocation

In [ ]:
%%time

import json

response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/x-image",
    TargetModel="resnet_18.tar.gz",  # this is the rest of the S3 path where the model artifacts are located
    Body=payload,
)

print(*json.loads(response["Body"].read()), sep="\n")


In [62]:
sm_client.delete_endpoint(EndpointName=endpoint_name)


{'ResponseMetadata': {'RequestId': 'f625e9bf-7e85-4c91-b3a6-5a608fce21e1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f625e9bf-7e85-4c91-b3a6-5a608fce21e1',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Thu, 10 Nov 2022 22:56:03 GMT'},
  'RetryAttempts': 0}}

In [63]:
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_name)
sm_client.delete_model(ModelName=endpoint_name)

{'ResponseMetadata': {'RequestId': 'db551d80-f234-4c46-8989-bfb71b14802a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'db551d80-f234-4c46-8989-bfb71b14802a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Thu, 10 Nov 2022 22:56:03 GMT'},
  'RetryAttempts': 0}}

## END 

<p>ImageDataGenerator generates batches of tensor image data with real-time data augmentation. 
The data will be looped over (in batches).

### Create a Multi-Model Endpoint

#### a) Copy Trained Models to a common S3 Prefix

In [ ]:
output_2 = f's3://{BUCKET}/{PREFIX}/mme/sign-language.tar.gz'

In [ ]:
print(output_2)

In [ ]:
# Create 100 copies of this model to simulate multiple models

import sagemaker

sess = sagemaker.Session()
s3_bucket = sess.default_bucket()  # Replace with your own bucket name if needed
print(s3_bucket)


In [ ]:
# Create 100 copies of this model to simulate multiple models

import sagemaker

sess = sagemaker.Session()
s3_bucket = sess.default_bucket()  # Replace with your own bucket name if needed
print(s3_bucket)

# Cell 06
import boto3

s3 = boto3.client("s3")
with open("models/model2/model.tar.gz", "rb") as f:
    # - s3://{BUCKET}/{PREFIX}/mme/sign-language.tar.gz
    s3.upload_fileobj(f, s3_bucket, f"{PREFIX}/mme/sign-language.tar.gz")

for i in range(0, 5):
    with open("models/model2/model.tar.gz", "rb") as f:
        # - s3://{BUCKET}/{PREFIX}/mme/sign-language-0.tar.gz
        s3.upload_fileobj(f, s3_bucket, f"{PREFIX}/mme/sign-language-{i}.tar.gz")

print("Models:uploaded and ready for use")
!aws s3 ls s3://{s3_bucket}/{PREFIX}/mme/
!aws s3 ls {output_2}

#### b) Essentials

In [ ]:
IMAGE_URI = '763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.3.1-cpu-py37-ubuntu18.04'
model_data_prefix = f's3://{BUCKET}/{PREFIX}/mme/'
model_data_prefix

In [ ]:
!aws s3 ls {model_data_prefix}

#### Deploy multi model end point

In [ ]:
# Cell 12
import boto3
from sagemaker import get_execution_role

sm_client = boto3.client(service_name="sagemaker")
runtime_sm_client = boto3.client(service_name="sagemaker-runtime")

account_id = boto3.client("sts").get_caller_identity()["Account"]
region = boto3.Session().region_name

role = get_execution_role()
print(role)

#### Create Model

model_url must point to the S3 prefix under which the models are stored so like with the "/" at end

s3://s3_bucket/cv-models/mme/

In [ ]:
# Cell 13
from time import gmtime, strftime

model_name = "mme-tensorflow-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
container = '763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.3.1-cpu-py37-ubuntu18.04'

instance_type = "ml.m5.xlarge"

print("Model name: " + model_name)
print("Model data Url: " + model_data_prefix)
print("Container image: " + container)

container = {"Image": container, "ModelDataUrl": model_data_prefix, "Mode": "MultiModel"}

create_model_response = sm_client.create_model(
    ModelName=model_name, ExecutionRoleArn=role, Containers=[container]
)

print("Model ARN: " + create_model_response["ModelArn"])

#### Create end point config

In [ ]:
# Cell 14
endpoint_config_name = model_name # - keep the name the same across for ease of tracking
print("Endpoint config name: " + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": instance_type,
            "InitialInstanceCount": 1,
            "InitialVariantWeight": 1,
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint config ARN: " + create_endpoint_config_response["EndpointConfigArn"])

#### Create end point

In [ ]:
%%time
# Cell 15

import time

endpoint_name = model_name # - keep the name the same across for ease of tracking
print("Endpoint name: " + endpoint_name)

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Endpoint Status: " + status)

print("Waiting for {} endpoint to be in service...".format(endpoint_name))
waiter = sm_client.get_waiter("endpoint_in_service")
waiter.wait(EndpointName=endpoint_name)

print("Created {} endpoint is in Service and read to invoke ...".format(endpoint_name))

#### Invoke the end point

In [ ]:
%matplotlib inline
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing import image
from IPython.display import Image
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
test_path  = './data/sign_language/test'
img = mpimg.imread(f'{test_path}/0/IMG_4159.JPG')
plt.imshow(img)

In [ ]:
import numpy as np
from PIL import Image

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def get_sample_image(img_path):
    img = Image.open(img_path).convert("RGB")
    #img = image.load_img(img_path, target_size=(224, 224))
    img = img.resize((224, 224))
    img = (np.array(img).astype(np.float32) / 255) - np.array(
        [0.485, 0.456, 0.406], dtype=np.float32
    ).reshape(1, 1, 3)
    img = img / np.array([0.229, 0.224, 0.225], dtype=np.float32).reshape(1, 1, 3)
    img = np.transpose(img, (2, 0, 1))
    return img.tolist()


In [ ]:
data = get_sample_image(f'{test_path}/0/IMG_4159.JPG')
print(len(data[0]))
print(type(data))
print(len(data))

In [ ]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer
from sagemaker.deserializers import JSONDeserializer

predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sess,
    serializer=CSVSerializer(), 
    deserializer=JSONDeserializer(),#C SVDeserializer(),
    #target_model="sign-language-0.tar.gz"
)
predictor.predict(data=data, target_model="sign-language-0.tar.gz")

In [ ]:
#with open(f'{test_path}/0/IMG_4159.JPG', "rb") as f:
with open(f'{test_path}/0/Prakash.jpeg', "rb") as f:    
    payload = f.read()
    
import json

response = sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType="application/x-image", 
    Body=payload,
    TargetModel="sign-language-0.tar.gz"
)
result = response["Body"].read()
# result will be in json format and convert it to ndarray
result = json.loads(result)
result

In [ ]:
result

In [ ]:
data = path_to_tensor(f'{test_path}/0/IMG_4159.JPG')
payload = {'instances': data}
payload
print(type(payload))
print(type(data))


In [ ]:
print(type(data))
data.shape


In [ ]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer

predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sess,
    serializer=CSVSerializer(), 
    deserializer=CSVDeserializer(),
    #target_model="sign-language-0.tar.gz"
)
predictor.predict(data, target_model="sign-language-0.tar.gz")

In [ ]:
sm_runtime = boto3.Session().client('sagemaker-runtime')

with open(f'{test_path}/0/IMG_4159.JPG', 'rb') as f:
    payload = f.read()
    payload = bytearray(payload)
    print(type(payload))
    print(payload)
    response = sm_runtime.invoke_endpoint(
        EndpointName=endpoint_name, 
        ContentType="application/x-image", 
        Body=payload,
        TargetModel="sign-language-0.tar.gz"
    )
    result = response["Body"].read()
    # result will be in json format and convert it to ndarray
    result = json.loads(result)
    # the result will output the probabilities for all classes
    # find the class with maximum probability and print the class index
    index = np.argmax(result)
    print("Result: label - " + object_categories[index] + ", probability - " + str(result[index]))

In [ ]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer

predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sess,
    serializer=CSVSerializer(), 
    deserializer=CSVDeserializer()
)
predictor.predict(data)


In [ ]:
# Cell 16
from datetime import datetime
import time
import json

data = np.array(data)
payload = json.dumps(data.tolist())

print(type(payload))

for i in range(0, 2):
    start_time = datetime.now()
    response = runtime_sm_client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType='application/json',
        TargetModel=f"sign-language-{i}.tar.gz",
        Body=payload
    )
    result = json.loads(response['Body'].read().decode())
    #res = result['predictions']
    time_delta = (datetime.now()-start_time).total_seconds() * 1000 
    time_delta = "{:.2f}".format(time_delta)
    
    print(f'Time={time_delta} --- > ::{len(result)}')

In [ ]:
result.keys()

In [ ]:
print(response.keys())
response['Body'].read().decode()
response['ResponseMetadata']